<a href="https://colab.research.google.com/github/DavidMihalcea880/GDP2023/blob/Maisy/array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1139, done.
remote: Total 1139 (delta 0), reused 0 (delta 0), pack-reused 1139
Receiving objects: 100% (1139/1139), 70.32 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (512/512), done.


In [ ]:
%cd yolov7

In [ ]:
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt -o yolov7-w6-pose.pt

In [ ]:
import torch
from torchvision import transforms
from torch import nn, optim
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts,plot_one_box
from utils.torch_utils import select_device
from models.experimental import attempt_load
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import os
import numpy as np
import json
%matplotlib inline

In [ ]:
def load_model():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()
    return model

model = load_model()

In [ ]:
@torch.no_grad()
def video_output(frame):
    image = frame
    # Apply transforms
    image = transforms.ToTensor()(image) 
    image = image.type(torch.cuda.FloatTensor)
    # Turn image into batch
    image = image.unsqueeze(0) 
    output, _ = model(image)
    output = non_max_suppression_kpt(output, 
                                     0.4, # Confidence Threshold
                                     0.7, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
    
    return  output,nimg
def listout(vid):
  cap = cv2.VideoCapture(vid)
  test=[]
  # Get the total number of frames in the video
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  # Round total_frames down to nearest multiple of 30
  rounded_frames = total_frames // 30 * 30
  # Generate equally spaced indices for the frames
  frame_indices = np.linspace(0, total_frames-1, rounded_frames, dtype=np.int32)
  for i, idx in enumerate(frame_indices):
      cap.set(cv2.CAP_PROP_POS_FRAMES, idx)  # Set the frame index
      ret, frame = cap.read()  # Read the frame
      if ret:  # If the frame is valid
        frame = letterbox(frame, 640, stride=64, auto=True)[0]
        output, nimg = video_output(frame)
        output=output[:,2:]
        test.append(output)
  return test

In [ ]:
!unzip /content/drive/MyDrive/violence-detection-dataset.zip

In [ ]:
test = listout('/content/yolov7/violence-detection-dataset/violent/1.mp4')

In [ ]:
shape=max(np.shape(x) for x in test)

In [ ]:
x=np.empty((120,2,56))
y=np.empty([])

In [ ]:
# Loop over each item in the list and extract the arrays
for i, item in enumerate(test):
    for j in range(shape[0]):
        x[i, j] = item[j]
y = [1]*shape[0]

It works for 1 video. But not all video has frame count 120 and 2 people. So maybe using array isn't a good choice. but I don't know how to operate a list or other python data type.

In [ ]:
mp4_dir = '/content/yolov7/violence-detection-dataset/violent'

# Get a list of all .avi files in the directory
mp4_files = [f for f in os.listdir(avi_dir) if f.endswith('.mp4')]

# Loop through the list of file names and pass each name to the listout function
for mp4_file in mp4_files:
    video_name =  os.path.join(mp4_dir, mp4_file)
    kp=listout(video_name)
    shape=max(np.shape(x) for x in kp)
    # Loop over each item in the list and extract the first and second arrays
for i, item in enumerate(test):
    x[i, 0] = item[0]
    x[i, 1] = item[1]
y = [1]*shape[0]